<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/SECTOR/CPU_SECTOR_ClimateBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 21:38:24] [setup] RAM Tracking...
[codecarbon INFO @ 21:38:24] [setup] GPU Tracking...
[codecarbon INFO @ 21:38:24] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:38:24] [setup] CPU Tracking...
[codecarbon WARNING @ 21:38:24] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:38:26] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 21:38:26] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:38:26] >>> Tracker's metadata:
[codecarbon INFO @ 21:38:26]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 21:38:26]   Python version: 3.10.12
[codecarbon INFO @ 21:38:26]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 21:38:26]   Available RAM : 12.675 GB
[codecarbon INFO @ 21:38:26]   CPU count: 2
[codecarbon INFO @ 21:38:26]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:38:26]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Manual Inputs


In [5]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"



#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "climatebert/distilroberta-base-climate-f"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "SECTOR-multilabel-climatebert"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=9.07e-5   #4.04e-5
batch_size = 16
num_train_epochs= 7
weight_decay=0.01
warmup_steps = 300
gradient_accumulation_steps = 1

# Automated

## Dataset

In [8]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

train_df = train_df.dropna(subset = ['SectorLabel']).reset_index(drop=True)
test_df = test_df.dropna(subset = ['SectorLabel']).reset_index(drop=True)
sector_list = [val  for list_ in list(train_df.SectorLabel) for val in list_]
sector_list = list(set(sector_list))

### Define labels ##################
label_names= sector_list
# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

for sector in sector_list:
    train_df[sector] = train_df.SectorLabel.apply(lambda x: True if sector in x else False)
    test_df[sector] = test_df.SectorLabel.apply(lambda x: True if sector in x else False)

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

{0: 'Agriculture', 1: 'Buildings', 2: 'Coastal Zone', 3: 'Cross-Cutting Area', 4: 'Disaster Risk Management (DRM)', 5: 'Economy-wide', 6: 'Education', 7: 'Energy', 8: 'Environment', 9: 'Health', 10: 'Industries', 11: 'LULUCF/Forestry', 12: 'Social Development', 13: 'Tourism', 14: 'Transport', 15: 'Urban', 16: 'Waste', 17: 'Water'}


In [9]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10123
0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207

 Classes Representation in Test Dataset: 936
0 . Agriculture : 200
1 . Buildings : 18
2 . Coastal Zone : 71
3 . Cross-Cutting Area : 180
4 . Disaster Risk Management (DRM) : 85
5 . Economy-wide : 85
6 . Education : 23
7 . Energy : 254
8 . Environment : 91
9 . Health : 68
10 . Industries : 41
11 . LULUCF/Forestry : 193
12 . Social Development : 56
13 . Tourism : 28
14 . Transport : 107
15 . Urban : 51
16 . Waste : 59
17 . Water : 106


In [10]:
# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}

for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))
    pos = sum(train_df[label])
    neg = len(train_df) - sum(train_df[label])
    positive_weights[label] = max(neg/pos, 1.1) if (neg/pos) < 10.0 else 10.0
    negative_weights[label] = min(pos/neg,1.0)
print(positive_weights)
print(negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207
{'Agriculture': 3.5293064876957496, 'Buildings': 10.0, 'Coastal Zone': 10.0, 'Cross-Cutting Area': 4.463032919589854, 'Disaster Risk Management (DRM)': 10.0, 'Economy-wide': 10.0, 'Education': 10.0, 'Energy': 2.5556726378644186, 'Environment': 10.0, 'Health': 10.0, 'Industries': 10.0, 'LULUCF/Forestry': 4.439548629768941, 'Social Development': 10.0, 'Tourism': 10.0, 'Transport': 7.63000852514919, 'Urban': 10.0, 'Waste': 10.0, 'Water': 7.386909693454847}
{'Agriculture': 0.283341784989858, 'Buildings': 0.01697809925658027, 'Coastal Zone': 0.07405835543766578, 'Cross-Cutting Area': 0.22

`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [11]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

[codecarbon INFO @ 21:46:41] Energy consumed for RAM : 0.000653 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 21:46:41] Energy consumed for all GPUs : 0.001364 kWh. Total GPU Power : 16.68245000108252 W
[codecarbon INFO @ 21:46:41] Energy consumed for all CPUs : 0.005843 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:46:41] 0.007860 kWh of electricity used since the beginning.


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 10123
Validation data: 936


Map:   0%|          | 0/10123 [00:00<?, ? examples/s]

[codecarbon INFO @ 21:46:56] Energy consumed for RAM : 0.000673 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 21:46:56] Energy consumed for all GPUs : 0.001477 kWh. Total GPU Power : 26.778300995154908 W
[codecarbon INFO @ 21:46:56] Energy consumed for all CPUs : 0.006022 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:46:56] 0.008172 kWh of electricity used since the beginning.


Map:   0%|          | 0/936 [00:00<?, ? examples/s]

Map:   0%|          | 0/10123 [00:00<?, ? examples/s]

Map:   0%|          | 0/936 [00:00<?, ? examples/s]

## Metrics

In [14]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [15]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [16]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights, reduction='mean')
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Training

In [17]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 21:47:22] [setup] RAM Tracking...
[codecarbon INFO @ 21:47:22] [setup] GPU Tracking...
[codecarbon INFO @ 21:47:22] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:47:22] [setup] CPU Tracking...
[codecarbon WARNING @ 21:47:22] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:47:23] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 21:47:23] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:47:23] >>> Tracker's metadata:
[codecarbon INFO @ 21:47:23]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 21:47:23]   Python version: 3.10.12
[codecarbon INFO @ 21:47:23]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 21:47:23]   Available RAM : 12.675 GB
[codecarbon INFO @ 21:47:23]   CPU count: 2
[codecarbon INFO @ 21:47:23]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 21:47:23]   GPU c

In [ ]:
multi_trainer.train()

In [19]:
tracker.stop()

[codecarbon INFO @ 22:29:48] Energy consumed for RAM : 0.004065 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 22:29:48] Energy consumed for all GPUs : 0.050455 kWh. Total GPU Power : 35.130100269538104 W
[codecarbon INFO @ 22:29:48] Energy consumed for all CPUs : 0.036374 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:29:48] 0.090894 kWh of electricity used since the beginning.


0.03174262746997449

In [25]:
multi_trainer.push_to_hub()

events.out.tfevents.1709675245.c0ef1fef2fb5.460.0:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/SECTOR-multilabel-climatebert/commit/fa567cefcf2e01355159d73acb0391a9a4b6061c', commit_message='End of training', commit_description='', oid='fa567cefcf2e01355159d73acb0391a9a4b6061c', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.6978,6.591099,0.000089,1.0,633,0.596817,0.394798,0.527409,0.498171,0.787296,0.867457,0.787296,0.525886,0.599587,0.579273,11.6320,80.468,5.072
1,0.4850,4.184384,0.000079,2.0,1266,0.525545,0.508915,0.636470,0.546874,0.798368,0.874944,0.798368,0.621597,0.690672,0.638373,11.8485,78.997,4.980
2,0.3657,2.752810,0.000061,3.0,1899,0.524821,0.498395,0.661720,0.539700,0.814103,0.876892,0.814103,0.618278,0.706554,0.639272,11.8724,78.838,4.970
3,0.2585,3.282129,0.000040,4.0,2532,0.545704,0.580727,0.714807,0.599207,0.800699,0.875203,0.800699,0.673199,0.744913,0.681324,12.0689,77.555,4.889
4,0.1841,13.732589,0.000019,5.0,3165,0.555084,0.601590,0.742563,0.619164,0.793706,0.867656,0.793706,0.684422,0.759028,0.691704,11.9715,78.186,4.928
5,0.1359,3.212297,0.000005,6.0,3798,0.591295,0.634906,0.750556,0.644929,0.784382,0.867633,0.784382,0.701773,0.766734,0.705670,12.0281,77.818,4.905
6,0.1133,4.047848,0.000000,7.0,4431,0.602760,0.639462,0.754345,0.647484,0.776224,0.858278,0.776224,0.701237,0.765498,0.704133,12.0522,77.662,4.895


# Predict and Evaluate

## Trainer

In [21]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()
y_true = y_true.bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

Agriculture
[[670  66]
 [ 30 170]] 

Buildings
[[910   8]
 [  4  14]] 

Coastal Zone
[[823  42]
 [ 17  54]] 

Cross-Cutting Area
[[650 106]
 [ 40 140]] 

Disaster Risk Management (DRM)
[[806  45]
 [ 26  59]] 

Economy-wide
[[788  63]
 [ 31  54]] 

Education
[[904   9]
 [  9  14]] 

Energy
[[634  48]
 [ 41 213]] 

Environment
[[795  50]
 [ 27  64]] 

Health
[[847  21]
 [ 17  51]] 

Industries
[[876  19]
 [  4  37]] 

LULUCF/Forestry
[[665  78]
 [ 30 163]] 

Social Development
[[854  26]
 [ 18  38]] 

Tourism
[[895  13]
 [ 12  16]] 

Transport
[[794  35]
 [ 25  82]] 

Urban
[[844  41]
 [ 22  29]] 

Waste
[[850  27]
 [  7  52]] 

Water
[[776  54]
 [ 24  82]] 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.720339,0.850000,0.779817,200.0
1,Buildings,0.636364,0.777778,0.700000,18.0
2,Coastal Zone,0.562500,0.760563,0.646707,71.0
3,Cross-Cutting Area,0.569106,0.777778,0.657277,180.0
4,Disaster Risk Management (DRM),0.567308,0.694118,0.624339,85.0
5,Economy-wide,0.461538,0.635294,0.534653,85.0
6,Education,0.608696,0.608696,0.608696,23.0
7,Energy,0.816092,0.838583,0.827184,254.0
8,Environment,0.561404,0.703297,0.624390,91.0
9,Health,0.708333,0.750000,0.728571,68.0
